In [1]:
# autoreload
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory of src to the path
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from src.db.manager import DBManager
from src.input_to_instructions.load_and_execute import *
from src.input_to_instructions.types import *
from src.plot_graph.execute import *
from src.operation.execute import *

In [3]:
from collections import defaultdict, Counter
import logging

import pandas as pd
import numpy as np
from tqdm import tqdm
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

import json
import itertools

from db.manager import DBManager
from operation.execute import OperationExecutor
from pathlib import Path
import warnings
import datetime


warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

INFO:db.instance:Connected to the database PerSite_DB


In [4]:
BASE_DIR = "../"
def read_json(path):
    with open(path, "r", encoding="utf-8") as f:
        result = json.loads(f.read())
    
    # result = [{"Input": d["Input"], "Response": json.dumps(d["Response"], ensure_ascii=False)} for d in result]
    return result

In [5]:
import matplotlib.pyplot as plt
import matplotlib

from src.plot_graph.execute import plot_graph_plotly
matplotlib.rcParams['font.family'] = 'NanumGothicCoding'
def run_query(user_input, metadata, instructions):
    variables = {
        "Metadata": metadata,
    }
    for instruction in instructions:
        # logger.debug(f"Executing instruction: {instruction.__class__.__name__}")
        # print(f"Executing instruction: {instruction.__class__.__name__}")
        
        if type(instruction) == InstructionQ:
            # Execute query
            result_df = DBManager.structured_query_data_t(metadata, instruction.args)
            
            if result_df is None:
                print("죄송합니다, 관련 데이터를 찾을 수 없습니다.", "response")
                return

            # For demo, drop rows where any value is -1
            result_df = result_df.loc[(result_df != -1).all(axis=1)]
           
            #pd.set_option('display.max_rows', 10000)        
            #pd.set_option('display.max_columns', 1000)
            #pd.set_option('display.width', 1000)
            #pd.set_option('display.max_colwidth', 1000)
            #print(f"QueryResult: {result_df}")

            variables[instruction.result_name] = result_df
        
        elif type(instruction) == InstructionO:
            # Execute operation

            result_dict = OperationExecutor.execute(variables, instruction.scripts, instruction.returns)
            
            variables.update(result_dict)
            pass
        elif type(instruction) == InstructionG:

            # fig = plot_graph(instruction, variables)
            # plt.show(fig)
            
            #fig = plot_graph_plotly(instruction, variables)
            #plt.show(fig)
            #fig.show()
            #print(type(fig))
            # print(fig, "graph")
            pass
        elif type(instruction) == InstructionR:
            #pass
            # # Execute response generation
            variables_to_report = {k: v for k, v in variables.items() if k not in ["Metadata"]}
            print(f"Variables: {variables_to_report}")
         
            
            
            # response, required_variables = ResponseGeneration.execute(instruction, variables, user_input, metadata)
            # print(f"Required variables: {required_variables}")
            
            # print(response, "response")

In [142]:
def build_query_groundtruth(dateset_name):
    def read(path):
        data = read_json(path)
        for i, d in enumerate(data):
            data[i]["Scenario"] = directory.name
            if "v7" in dateset_name:
                data[i]["Metadata"] = metadata
        return data

    ds_ts = []
    ds_tr = []
    base_dataset_dir = Path(f"{BASE_DIR}/finetuning/dataset/{dateset_name}")
    
    for directory in base_dataset_dir.iterdir():
        if directory.is_dir() and "scenario1" in directory.name:
            if "v7" in dateset_name:
                metadata = read_json(f"{directory}/metadata.json")
                
            
            ds_ts.extend(read(f"{directory}/onlyq_ts.json"))
            #ds_tr.extend(read(f"{directory}/onlyq_ts.json"))
            #ds_tr.extend(read(f"{directory}/graph_temp.json"))
    
    ds = ds_ts + ds_tr
    
    # if "v7" in dateset_name:
    #     db_gt_filename = f"{BASE_DIR}/experiments/db_gt_v7.json"
    # else:
    #     db_gt_filename = f"{BASE_DIR}/experiments/db_gt.json"
    #     metadata = None
    
    # with open(db_gt_filename, "w", encoding="utf-8") as f:
        # f.write("[")
    # with tqdm(total=len(ds)) as pbar:
    #count=0
    for d in ds:
        #count += 1
        #if count < 10:
        #    continue  # 10보다 작은 값은 건너뛰기
        #if count > 40:
        #    break     # 20을 초과하면 반복 종료
        # pbar.set_description(f"Processing {d['Input']}")
        # print("--")
        
        instructions = InputToInstruction.postprocess(d['Response'])
        user_input, tags, metadata, scenario = d["Input"], d["Tags"], d["Metadata"], d["Scenario"]
        print(user_input)
        
        run_query(user_input, metadata, instructions)
        
  

In [143]:

build_query_groundtruth("v7-250309-reduceinputanddatefunctioncall")

Why is our classroom so cold
Variables: {'qr':      roomtemp  settemp   oper            timestamp     idu
0        28.0     23.0  False  2022-09-30 09:00:00  01_IB5
1        28.0     23.0  False  2022-09-30 09:01:00  01_IB5
2        28.0     23.0  False  2022-09-30 09:02:00  01_IB5
3        28.0     23.0  False  2022-09-30 09:03:00  01_IB5
4        28.0     23.0  False  2022-09-30 09:04:00  01_IB5
..        ...      ...    ...                  ...     ...
173      28.0     23.0  False  2022-09-30 11:56:00  01_IB5
174      28.0     23.0  False  2022-09-30 11:57:00  01_IB5
175      28.0     23.0  False  2022-09-30 11:58:00  01_IB5
176      28.0     23.0  False  2022-09-30 11:59:00  01_IB5
177      28.0     23.0  False  2022-09-30 12:00:00  01_IB5

[178 rows x 5 columns]}
오늘 아침과 저녁의 온도차이는 얼마나 돼?
Variables: {'qr_morning':      roomtemp           timestamp     idu
0        26.5 2022-09-30 06:00:00  01_IB5
1        26.5 2022-09-30 06:01:00  01_IB5
2        26.5 2022-09-30 06:02:00  01_IB5
3 

In [ ]:
# run_scenarios.py
# -*- coding: utf-8 -*-
"""
scenario1/onlyq_tr.json 들을 순회하며
 1) SQL_Q  실행 → DataFrame 출력
 2) SQL_O  실행 → DataFrame (또는 row 수) 출력
"""

import json, textwrap, re
from pathlib import Path
import pandas as pd


# ──────────────────────────────────────────
BASE_DIR     = "/workspace"
dateset_name = "v7-250309-reduceinputanddatefunctioncall"
DATE_FIXES = {
    'CURRENT_DATE':      "DATE '2022-09-30'",
    'CURRENT_TIMESTAMP': "TIMESTAMP '2022-09-30 12:00:00'"
}
#DATE_FIXES = {
#    'CURRENT_DATE':      "DATE '2022-09-17'",
#    'CURRENT_TIMESTAMP': "TIMESTAMP '2022-09-17 16:30:00'"
#}


# ──────────────────────────────────────────

def apply_date_fixes(sql: str) -> str:
    for k, v in DATE_FIXES.items():
        sql = re.sub(rf'\b{k}\b', v, sql)
    return sql

import re, textwrap
import re, textwrap
def wrap_into_select(sql_o: str, sql_q: str) -> str:
    # ①  /* SQL_Q */ 매크로 치환만 수행
    if '/* SQL_Q */' in sql_o:
        sql_o = sql_o.replace('/* SQL_Q */', textwrap.dedent(sql_q).rstrip(';'))

    # ②  불필요한 ORDER BY · LIMIT 강제 삽입 로직 제거
    sql_o = textwrap.dedent(sql_o).strip().rstrip(';')

    # ③  SELECT 로 시작하지 않으면 서브쿼리로 래핑
    if not sql_o.lstrip().upper().startswith('SELECT'):
        sql_o = f"SELECT * FROM (\n{sql_o}\n) AS sub"

    return sql_o + ';'

def read_json(p: Path):
    with open(p, encoding='utf-8') as f:
        return json.load(f)

# ───────────────── 데이터 적재 ──────────────────
base_dataset_dir = Path(f"{BASE_DIR}/finetuning/dataset/{dateset_name}")
scenarios = []
for directory in base_dataset_dir.iterdir():
    if directory.is_dir() and "scenario1" in directory.name:
        if "v7" in dateset_name:
            _metadata = read_json(directory / "metadata.json")
        scenarios.extend(read_json(directory / "trsql.json"))

# ───────────────── 실행 루프 ────────────────────
for idx, block in enumerate(scenarios, 1):
    inp, tags = block['Input'], block['Tags']
    sql_q = apply_date_fixes(tags['SQL_Q'])
    sql_o = apply_date_fixes(tags['SQL_O'])

    print(f"\n▶ ({idx}) {inp}")

    # ---------- ① SQL_Q 실행 ----------
    try:
        rows_q = DBManager.execute_sql(sql_q)
        desc_q = DBManager.db_instance.cursor.description
        if desc_q:                                     # SELECT 결과가 있을 때만
            cols_q = [d[0] for d in desc_q]
            df_q   = pd.DataFrame(rows_q, columns=cols_q)
            print("\n― SQL_Q 결과 ―")
            print(df_q if not df_q.empty else '(empty result)')
        else:
            print("\n― SQL_Q 결과 없음 (non-select) ―")
    except Exception as e:
        print(f"SQL_Q ERROR → {e}")

    # ---------- ② SQL_O 실행 ----------
    try:
        final_sql = wrap_into_select(sql_o, sql_q)
        rows_o    = DBManager.execute_sql(final_sql)
        desc_o    = DBManager.db_instance.cursor.description
        if desc_o:
            cols_o = [d[0] for d in desc_o]
            df_o   = pd.DataFrame(rows_o, columns=cols_o)
            print("\n― SQL_O 결과 ―")
            print(df_o if not df_o.empty else '(empty result)')
        else:
            print(f"\n― SQL_O 수행 완료 (행 {DBManager.db_instance.cursor.rowcount}개, 반환 컬럼 없음) ―")
    except Exception as e:
        print(f"SQL_O ERROR → {e}")

DBManager.db_instance.close()


INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched



▶ (1) 어제 우리반과 옆반의 설정온도 차이 알려줘

― SQL_Q 결과 ―
      settemp idu_name           timestamp
0        23.0   01_IB5 2022-09-16 00:00:00
1        23.0   01_IB7 2022-09-16 00:00:00
2        23.0   01_IB5 2022-09-16 00:01:00
3        23.0   01_IB7 2022-09-16 00:01:00
4        23.0   01_IB5 2022-09-16 00:02:00
...       ...      ...                 ...
2849     23.0   01_IB7 2022-09-16 23:57:00
2850     23.0   01_IB5 2022-09-16 23:58:00
2851     23.0   01_IB7 2022-09-16 23:58:00
2852     23.0   01_IB5 2022-09-16 23:59:00
2853     23.0   01_IB7 2022-09-16 23:59:00

[2854 rows x 3 columns]

― SQL_O 결과 ―
   settemp_ours  settemp_beside  settemp_diff
0          23.0            23.0           0.0

▶ (2) 오늘 우리반과 옆반의 평균 온도차이 알려줘

― SQL_Q 결과 ―
      roomtemp idu_name           timestamp
0         26.0   01_IB5 2022-09-17 00:00:00
1         25.0   01_IB7 2022-09-17 00:00:00
2         25.5   01_IB5 2022-09-17 00:01:00
3         25.0   01_IB7 2022-09-17 00:01:00
4         25.5   01_IB5 2022-09-17 00:02:00

INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched



― SQL_Q 결과 ―
        roomtemp           timestamp
0           -1.0 2021-04-01 00:00:00
1           -1.0 2021-04-01 00:01:00
2           -1.0 2021-04-01 00:02:00
3           -1.0 2021-04-01 00:03:00
4           -1.0 2021-04-01 00:04:00
...          ...                 ...
131035      -1.0 2021-06-30 23:55:00
131036      -1.0 2021-06-30 23:56:00
131037      -1.0 2021-06-30 23:57:00
131038      -1.0 2021-06-30 23:58:00
131039      -1.0 2021-06-30 23:59:00

[131040 rows x 2 columns]

― SQL_O 결과 ―
   roomtemp_avg
0          -1.0

▶ (4) 올해 여름 앞반 평균온도 알려줘


INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched


...


― SQL_O 결과 ―
                                               dates
0  [2022-09-18, 2022-09-13, 2022-09-16, 2022-09-1...

▶ (49) 어제 전원 껐어?

― SQL_Q 결과 ―
       oper           timestamp
0     False 2022-09-16 00:00:00
1     False 2022-09-16 00:01:00
2     False 2022-09-16 00:02:00
3     False 2022-09-16 00:03:00
4     False 2022-09-16 00:04:00
...     ...                 ...
1435  False 2022-09-16 23:55:00
1436  False 2022-09-16 23:56:00
1437  False 2022-09-16 23:57:00
1438  False 2022-09-16 23:58:00
1439  False 2022-09-16 23:59:00

[1440 rows x 2 columns]

― SQL_O 결과 ―
    oper
0  False

▶ (50) 최근에 설정온도가 가장 높았던 날 알려줘

― SQL_Q 결과 ―
       settemp           timestamp
0         23.0 2022-09-10 00:00:00
1         23.0 2022-09-10 00:01:00
2         23.0 2022-09-10 00:02:00
3         23.0 2022-09-10 00:03:00
4         23.0 2022-09-10 00:04:00
...        ...                 ...
11373     23.0 2022-09-17 23:55:00
11374     23.0 2022-09-17 23:56:00
11375     23.0 2022-09-17 23:57:00
11376     23

INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
ERROR:db.instance:An error occurred while executing SQL query: column "minv.coldest_temp" must appear in the GROUP BY clause or be used in an aggregate function
LINE 20:        coldest_temp
                ^




― SQL_Q 결과 ―
       roomtemp           timestamp
0          26.5 2022-08-01 00:01:00
1          26.5 2022-08-01 00:02:00
2          26.5 2022-08-01 00:03:00
3          26.5 2022-08-01 00:04:00
4          26.5 2022-08-01 00:05:00
...         ...                 ...
44075      25.0 2022-08-31 23:55:00
44076      25.0 2022-08-31 23:56:00
44077      25.0 2022-08-31 23:57:00
44078      25.0 2022-08-31 23:58:00
44079      25.0 2022-08-31 23:59:00

[44080 rows x 2 columns]

― SQL_O 결과 ―
   roomtemp_max  roomtemp_min
0          27.5          22.0

▶ (54) 지금 우리반 너무 더워

― SQL_Q 결과 ―
     roomtemp  settemp   oper           timestamp
0        25.5     23.0  False 2022-09-17 13:30:00
1        26.0     23.0  False 2022-09-17 13:31:00
2        26.0     23.0  False 2022-09-17 13:35:00
3        26.0     23.0  False 2022-09-17 13:36:00
4        26.0     23.0  False 2022-09-17 13:37:00
..        ...      ...    ...                 ...
165      26.0     23.0  False 2022-09-17 16:26:00
166      26.0     2

INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched
INFO:db.instance:SQL SELECT query executed successfully and results fetched



― SQL_Q 결과 ―
        settemp           timestamp
0          23.0 2022-08-17 00:00:00
1          23.0 2022-08-17 00:00:00
2          23.0 2022-08-17 00:00:00
3          23.0 2022-08-17 00:01:00
4          23.0 2022-08-17 00:01:00
...         ...                 ...
131938     23.0 2022-09-16 23:58:00
131939     23.0 2022-09-16 23:58:00
131940     23.0 2022-09-16 23:59:00
131941     23.0 2022-09-16 23:59:00
131942     23.0 2022-09-16 23:59:00

[131943 rows x 2 columns]

― SQL_O 결과 ―
   settemp_max  settemp_min
0         25.0         21.0

▶ (59) 오늘 4층에서 가장 더운 방은 어디야?

― SQL_Q 결과 ―
      roomtemp idu_name           timestamp
0         29.0   02_I85 2022-09-17 00:00:00
1         25.0   01_IB7 2022-09-17 00:00:00
2         28.5   02_I84 2022-09-17 00:00:00
3         29.0   02_I85 2022-09-17 00:01:00
4         25.0   01_IB7 2022-09-17 00:01:00
...        ...      ...                 ...
4255      25.5   01_IB7 2022-09-17 23:58:00
4256      26.5   02_I84 2022-09-17 23:58:00
4257      25.0   